# Toward Controlled Generation of Text

## Idea
The authors aim to disentangle representations of style and content in the latent code of an Adversarial Variational AutoEncoder. The style is called the 'structured code' and is learned by discriminators for each code attribute that needs to be disentangled from the latent space.

## Method
* This method does not use adversarial training.
* The authors describe 3 separate losses to train their model.
    * A reconstruction loss that ensures that the generated sentence $\hat{x}$ is the same as the original sentence $x$
    * A discriminator that classifies $\hat{x}$ and validates if the predicted class is the same as the corresponding labels for $x$

## Observations